## 1. Importing the necessary libraries
We first import the necessary dependencies for our analysis as well as our data for further processing.

The required libraries are imported below:

In [1]:
import os
import json
import nltk
import json
import pickle
import pandas as pd
import wordcloud as wc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import bertopic
import numpy as np
from typing import List
import spacy

import rich
try:
    import networkx as nx
    import graph_tool.all as gt
except ImportError as e:
    print(f"Error: {e}")
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from plotly import io as pio
from sentence_transformers import SentenceTransformer

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
nltk.download('stopwords')

os.chdir("/home/balint/ai-safety-ethics")

def get_authors(author_dict, fix_first_name=True):
    aths = []
    for author in author_dict:
        if "literal" in author:
            continue
        if author["given"] and fix_first_name:
            if not author["given"].isupper():
                # Turn author given name into initials
                author["given"] = "".join([n[0] for n in author["given"].split(" ")])
            name = author["family"] + ", " + ".".join(author["given"].replace(".", "")) + "."
        elif author["given"]:
            name = author["family"] + ", " + author["given"]
        else:
            name = author["family"]
        aths.append(name)
    return " and ".join(aths)


def model_topics(
        corpus: List[str],
        dates,
        path_to_data="output",
        fname="",
        min_cluster_size=3, rerun_embeddings=False, rerun_topic=True):
    embeddings_path = os.path.join(path_to_data, f"{fname}embeddings.p")
    if not os.path.exists(embeddings_path) or rerun_embeddings:
        sentence_model = SentenceTransformer("all-mpnet-base-v2")
        embeddings = sentence_model.encode(corpus, show_progress_bar=True)
        pickle.dump(embeddings, open(embeddings_path, "wb"))
    else:
        embeddings = pickle.load(open(embeddings_path, "rb"))

    topic_model_path = os.path.join(path_to_data, f"{fname}topic_model.p")
    if not os.path.exists(topic_model_path) or rerun_topic:
        ctfidf_model = bertopic.vectorizers.ClassTfidfTransformer(reduce_frequent_words=True)
        vectorizer_model = CountVectorizer(stop_words="english")
        hdbscan_model = HDBSCAN(
            min_cluster_size=min_cluster_size, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

        topic_model = bertopic.BERTopic(
            hdbscan_model=hdbscan_model,
            ctfidf_model=ctfidf_model,
            vectorizer_model=vectorizer_model,
            verbose=True)
        topic_model.fit(corpus, embeddings)
        topic_model.save(topic_model_path)
    else:
        topic_model = bertopic.BERTopic.load(topic_model_path)

    return topic_model, embeddings

[nltk_data] Downloading package stopwords to /home/balint/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We then set local variables to determine which corpus ('safety' or 'ethics') we want to use with what date cutoff. If using the ethics corpus, then we may also specify which conference to use.

In [2]:
# Set this variable to true if only want to process information for the titles but not the astracts. Otherwise, both the title and the abstract will be processed.
use_title_only = False
load_all = True  # If true then load all papers regardless of the below parameters

conference = 'both' # Set this variable to choose which conference proceedings to process. The options are 'aies', 'facct', and 'both'.
field = 'safety'  # Either 'safety' or 'ethics'
if load_all:
    field = "combined"
year_cutoff = None  # 2022
month_cutoff = None  # 11

assert conference in ['aies', 'facct', 'both'], "Invalid conference option. Choose 'aies', 'facct', or 'both'."
assert field in ['safety', 'ethics', 'combined'], "Invalid field option. Choose 'safety' or 'ethics'."

Now we load the data:

In [3]:
# Load both JSON files for the AIES and FAccT proceedings and extract either the title or the abstract and title from the file.
docs = []
dates = []
titles = []
types = []

if load_all:
    ethics = json.load(open('data/ethics/all_ethics.json', "r", encoding="utf-8"))
    safety = json.load(open('data/safety/all_safety.json', "r", encoding="utf-8"))
    data = ethics + safety
elif field == 'ethics':
    aies = json.load(open('data/ethics/AIES.json', "r", encoding="utf-8"))
    facct = json.load(open('data/ethics/FACCT.json', "r", encoding="utf-8"))
    data = aies + facct if conference == 'both' else aies if conference == 'aies' else facct
    safety = []
else:
    safety = json.load(open('data/safety/all_safety.json', "r", encoding="utf-8"))
    data = safety

split_index = None
used_ids = set()
for i, paper in enumerate(data):
    year = int(paper['issued']['date-parts'][0][0])
    month = int(paper['issued']['date-parts'][0][1]) if len(paper['issued']['date-parts'][0]) > 1 else 13

    if not year:
        print(f"Paper {paper['id']} has no year. Skipping.")

    if year_cutoff and year < year_cutoff:
        continue
    if month_cutoff and year == year_cutoff and month < month_cutoff:
        continue

    if paper['id'][-1] in ['a', 'b', 'c', 'd']:
        paper['id'] = paper['id'][:-1]
    pid = paper['id']
    title = paper['title'].lower()
    if title in map(str.lower, titles):
        print(f"Duplicate title: {paper['id']}")
    if pid in used_ids:
        print(f"Duplicate ID: {paper['id']}")
    used_ids.add(pid)

    titles.append(title)
    if use_title_only:
        docs.append(title)
        types.append("safety" if paper in safety else "ethics")
    elif not use_title_only and 'abstract' in paper:
        text = title + ' ' + paper['abstract'].lower()
        if "©" in text:
            text = text[:text.index("©")] # Remove copyright information
        docs.append(text)
        dates.append(year)
        types.append("safety" if paper in safety else "ethics")
    else:
        print(f"Paper {paper['id']} has no abstract. Skipping.")

    if len(types) > 2 and types[-1] != types[-2]:
        split_index = i

print(f"Number of papers: {len(docs)}")
print(f"Split index: {split_index}")
print(f"Number of safety papers: {len([t for t in types if t == 'safety'])}")
print(f"Number of ethics papers: {len([t for t in types if t == 'ethics'])}")
if load_all:
    print("Load all is true!")

# Create output folders
os.makedirs('output', exist_ok=True)
os.makedirs(f'output/{field}', exist_ok=True)

Number of papers: 2655
Split index: 1329
Number of safety papers: 1329
Number of ethics papers: 1326
Load all is true!


We preprocess the documents by removing URLs and symbols and then normalizing the entire text.

In [ ]:
# Save corpus for VOSviewer use

scores = ["0"] * len([t for t in types if t == 'safety']) + ["1"] * len([t for t in types if t == 'ethics'])
with open("data/scores.txt", "w") as f:
    f.write("\n".join(scores))
with open("data/corpus.txt", "w") as f:
    f.write("\n".join(map(lambda x: x.replace("\n", " "), docs)))

In [ ]:
import textacy
from functools import partial
from textacy import preprocessing


def postproc(doc: spacy.tokens.Doc) -> str:
    remove_list = ["textit", "emph", "ieee"]
    filtered = filter(lambda t: not t.is_stop and t not in nltk.corpus.stopwords.words("english") and t.lemma_ not in remove_list, doc)
    lemmatized = map(lambda t: t.lemma_, filtered)
    return list(lemmatized)

preproc = preprocessing.make_pipeline(
    partial(preprocessing.replace.urls, repl=""),
    partial(preprocessing.replace.currency_symbols, repl=""),
    preprocessing.remove.accents,
    preprocessing.remove.html_tags,
    preprocessing.remove.punctuation,
    preprocessing.normalize.unicode,
    preprocessing.normalize.whitespace,
    preprocessing.normalize.quotation_marks,
    preprocessing.normalize.hyphenated_words
)

rerun_corpus = False
if os.path.exists("output/combined/corpus.p") and not rerun_corpus:
    print("Loading corpus from file...")
    corpus = pickle.load(open("output/combined/corpus.p", "rb"))
else:
    try:
        spacy.require_gpu()
        print(f"Creating corpus using GPU...")
    except ValueError:
        print(f"Creating corpus using CPU...")
    corpus = textacy.Corpus("en_core_web_sm", map(preproc, docs))
    pickle.dump(corpus, open("output/combined/corpus.p", "wb"))

print("Running token post-processing...")
corpus_tokens = []
corpus_text = []
for doc in corpus:
    tokens = postproc(doc)
    corpus_tokens.append(tokens)
    corpus_text.append(" ".join(tokens))

Loading corpus from file...
Running token post-processing...


# 2. Topic analysis with embeddings and BERTopic
We will now perform topic analysis on the titles and abstracts of the papers using the BERTopic library.

This uses a utility function from the file `util.py` to embed the data, model topics, and save everything in the appropriate location. Each visualisation will also be saved as an interactive HTML file under the folder `output`.

In [ ]:
rerun_embeddings = False
min_cluster_size = 12
topic_model, embeddings = model_topics(
    corpus_text, [], path_to_data=f"output/{field}",
    min_cluster_size=min_cluster_size, rerun_embeddings=rerun_embeddings)
n_topics = len(topic_model.get_topics())
print(f"Number of topics: {n_topics}")
document_topics = topic_model.get_document_info(corpus_text)

2025-02-20 15:30:54,173 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-20 15:30:59,073 - BERTopic - Dimensionality - Completed ✓
2025-02-20 15:30:59,073 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-20 15:30:59,117 - BERTopic - Cluster - Completed ✓
2025-02-20 15:30:59,119 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-20 15:30:59,272 - BERTopic - Representation - Completed ✓
2025-02-20 15:30:59,415 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Number of topics: 39


In [48]:
min_cluster_size = 8
safety_corpus = corpus_text[:split_index]
safety_topic_model, safety_embeddings = model_topics(
    safety_corpus, [], path_to_data=f"output/{field}", fname="safety_",
    min_cluster_size=min_cluster_size, rerun_embeddings=rerun_embeddings)
n_topics = len(safety_topic_model.get_topics())
print(f"Number of safety topics: {n_topics}")
safety_document_topics = safety_topic_model.get_document_info(safety_corpus)

2025-02-20 15:30:59,537 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-20 15:31:00,915 - BERTopic - Dimensionality - Completed ✓
2025-02-20 15:31:00,916 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-20 15:31:00,939 - BERTopic - Cluster - Completed ✓
2025-02-20 15:31:00,941 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-20 15:31:01,033 - BERTopic - Representation - Completed ✓
2025-02-20 15:31:01,131 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Number of safety topics: 35


In [49]:
min_cluster_size = 8
ethics_corpus = corpus_text[split_index:]
ethics_topic_model, ethics_embeddings = model_topics(
    ethics_corpus, [], path_to_data=f"output/{field}", fname="ethics_",
    min_cluster_size=min_cluster_size, rerun_embeddings=rerun_embeddings)
n_topics = len(ethics_topic_model.get_topics())
print(f"Number of ethics topics: {n_topics}")
ethics_document_topics = ethics_topic_model.get_document_info(ethics_corpus)

2025-02-20 15:31:01,223 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-20 15:31:02,616 - BERTopic - Dimensionality - Completed ✓
2025-02-20 15:31:02,616 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-20 15:31:02,638 - BERTopic - Cluster - Completed ✓
2025-02-20 15:31:02,640 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-20 15:31:02,736 - BERTopic - Representation - Completed ✓
2025-02-20 15:31:02,863 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Number of ethics topics: 29


In [59]:
from sklearn.metrics.pairwise import cosine_similarity

shared_embeddings = True
most_similar = False  # Whether to find the most similar or least similar pairs

if shared_embeddings:
    sem = embeddings[:split_index]
    eem = embeddings[split_index:]
    dt = document_topics
else:
    sem = safety_embeddings
    eem = ethics_embeddings
    dt = pd.concat([safety_document_topics, ethics_document_topics], axis=0)

similarity = cosine_similarity(sem, eem)
if most_similar:
    large_sim = similarity.argsort(axis=None)[::-1]
    ixs = np.unravel_index(large_sim, similarity.shape)
else:
    low_sim = similarity.argsort(axis=None)
    ixs = np.unravel_index(low_sim, similarity.shape)

The following will genertae a table of the top N most similar or dissimilar documents between the AI safety and ethics corpora as measure by their cosine similarity:

In [60]:
n_limit = 50
included_ids = set()
ethics_df = []
safety_df = []
for safety_idx, ethics_idx in zip(ixs[0], split_index + ixs[1]):
    if len(safety_df) >= n_limit and len(ethics_df) >= n_limit:
        break

    safety_paper = data[safety_idx]
    if safety_paper['id'] not in included_ids and len(safety_df) < n_limit:
        paper = {
            "Included": True,
            "Ref": safety_paper["id"],
            "Title": safety_paper["title"],
            "Authors": get_authors(safety_paper["author"]),
            "Year": safety_paper["issued"]["date-parts"][0][0],
            "Field": "Safety",
            "Link": safety_paper.get("URL", "Unkown"),
            "DOI": safety_paper.get("DOI", "Unkown"),
            "Citations": 0,
            "Type": safety_paper.get("type", "Unknown")
        }
        safety_df.append(paper)
        included_ids.add(safety_paper['id'])

    ethics_paper = data[ethics_idx]
    if ethics_paper['id'] not in included_ids and len(ethics_df) < n_limit:
        paper = {
            "Included": True,
            "Ref": ethics_paper["id"],
            "Title": ethics_paper["title"],
            "Authors": get_authors(ethics_paper["author"]),
            "Year": ethics_paper["issued"]["date-parts"][0][0],
            "Field": "Ethics",
            "Link": ethics_paper.get("URL", "Unkown"),
            "DOI": ethics_paper.get("DOI", "Unkown"),
            "Citations": 0,
            "Type": ethics_paper.get("type", "Unknown")
        }
        ethics_df.append(paper)
        included_ids.add(ethics_paper['id'])

safety_df = pd.DataFrame(safety_df)
ethics_df = pd.DataFrame(ethics_df)
df = pd.concat([safety_df, ethics_df], axis=0)
fname = "most_similar" if most_similar else "least_similar"
df.to_csv(f"output/{fname}.csv", index=False)

In [62]:
get_authors([s for s in safety if s["id"] == "lightmanLetVerifyStep2023"][0]["author"])

'Lightman, H. and Kosaraju, V. and Burda, Y. and Edwards, H. and Baker, B. and Lee, T. and Leike, J. and Schulman, J. and Sutskever, I. and Cobbe, K.'

Run the following code snippet to generate a topic and document map:

In [ ]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
fig = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
# fig.write_html(f"output/{field}/documents-{conference}.html", include_mathjax = 'cdn')
fig.show()

Run the following code to visualise which tokens are most frequent under each topic:


In [ ]:
fig = topic_model.visualize_barchart(top_n_topics=12, n_words=6, title=f"{field.capitalize()}: Top 12 Topics and Keywords")
# fig.write_html(f"output/{field}/tokens-{conference}.html", include_mathjax = 'cdn')
fig.show()

The code below generates a document and topic map with much more pleasing aesthetics, however, it does not provide interactivity:


In [ ]:
import datamapplot

# title = "AIES and FAccT Topics and Documents" if field == 'ethics' else "Safety Topics and Documents"
title = "Map of AI Ethics Topics and Documents"
fig = topic_model.visualize_document_datamap(docs,
                                             title="",
                                            #  sub_title="A visualisation of document embeddings clustered by topic",
                                             embeddings=embeddings,
                                             label_over_points=True,
                                             dynamic_label_size=True,
                                             max_font_size=24,
                                             min_font_size=8,
                                             custom_labels=True
                                             )
fig.savefig(f"output/{field}/datamap.pdf", bbox_inches='tight', pad_inches=0.0)
fig.show()

# 3. Graph analysis

We build a graph of document similarities based on their embedding similarity.

In [ ]:
sim_thershold = 0.67
use_nx = False

edge_list = []
for i, row in enumerate(similarity):
    for j, col in enumerate(row):
        if col > sim_thershold:
            edge_list.append((i, j + split_index, col))

if use_nx:
    g = nx.Graph()
    g.add_weighted_edges_from(edge_list)
    print(f"Create graph with {g.number_of_nodes()} vertices and {g.number_of_edges()} edges.")
else:
    g = gt.Graph(directed=False)
    weight = g.new_edge_property("float")
    g.add_edge_list(edge_list, eprops=[weight], hashed=False)
    g.ep["weight"] = weight
    doc_id = g.new_vertex_property("int", range(g.num_vertices()))
    g.vp["doc_id"] = doc_id
    g.remove_vertex([v for v in g.vertices() if g.vertex(v).out_degree() == 0 and g.vertex(v).in_degree() == 0])
    print(f"Create graph with {g.num_vertices()} vertices and {g.num_edges()} edges.")

In [ ]:
from textacy.representations.network import rank_nodes_by_bestcoverage

if use_nx:
    ranking = rank_nodes_by_bestcoverage(g, c=1, alpha=0.1, k=len(g.nodes()))
    rich.print([data[i]["title"] for i in ranking][:10])
    nx.set_node_attributes(g, ranking, "rank")
else:
    vb, eb = gt.betweenness(g, weight=g.ep.weight)
    g.vp["vb"] = vb
    g.ep["eb"] = eb
    rich.print([data[i]["title"] for i in vb.fa.argsort()[::-1][:10]])

In [ ]:
if not use_nx:
    safety = g.new_vertex_property("bool")
    title = g.new_vertex_property("string")
    color = g.new_vertex_property("vector<double>")
    topic = g.new_vertex_property("int")
    topic_color = g.new_vertex_property("vector<double>")

    for i, v in enumerate(g.vertices()):
        # title[v] = data[i]["title"][:20]
        safety[v] = types[g.vp.doc_id[v]] == "safety"
        alpha = gt.prop_to_size(g.vp.vb, 0.2, 1, power=.1)
        color[v] = [0., 0.247, 0.361, alpha[v]] if types[g.vp.doc_id[v]] == "safety" \
            else [1., 0.388, 0.38, alpha[v]]
        topic[v] = 1 + dt.iloc[g.vp.doc_id[v]]["Topic"]
        topic_color[v] = plt.get_cmap("tab20").colors[topic[v] % 20]

    for i, v in enumerate(vb.fa.argsort()[::-1][:10]):
        title[v] = data[g.vp.doc_id[v]]["title"][:40] + "..."

    g.vp["safety"] = safety
    g.vp["title"] = title
    g.vp["color"] = color
    g.vp["topic"] = topic
    g.vp["topic_color"] = topic_color

    u = gt.extract_largest_component(g)
    pos = gt.sfdp_layout(u, eweight=g.ep.weight, groups=topic, gamma=.01, kc=10, r=3, C=0.5, max_iter=1e5)
    gt.graph_draw(u, pos, output="output/graph.png", bg_color="white", output_size=(800, 500), fit_view=True, adjust_aspect=False,
           vertex_size=gt.prop_to_size(g.vp.vb, 2, 15, power=.4), vorder=g.vp.vb, vertex_fill_color=g.vp.color, vertex_color=g.vp.color,
           vertex_text=title, vertex_text_color=color, vertex_text_position=0, vertex_text_rotation=0,
           vertex_font_size=10, vertex_text_out_color="black",
           vertex_text_out_width=0.005, edge_pen_width=gt.prop_to_size(g.ep.weight, 0.1, 0.5, power=3)
           )

In [ ]:
u.save("output/graph.graphml")
# np.unique_counts(safety.fa)

In [ ]:
if not use_nx:
    u = gt.extract_largest_component(g)
    gt.graph_draw(u, pos, output="output/graph_topic.png", bg_color="white", output_size=(1000, 1000), fit_view=True,
           vertex_size=gt.prop_to_size(g.vp.vb, 2, 15, power=.3), vorder=g.vp.vb, vertex_fill_color=g.vp.topic_color, vertex_color=g.vp.color,
           #vertex_text=titles, vertex_text_color=color, vertex_text_position=0, vertex_font_size=10,
           edge_pen_width=gt.prop_to_size(g.ep.weight, 0.1, 0.15, power=2)
           )

In [ ]:
if not use_nx:
    u = gt.extract_largest_component(g)
    state = gt.minimize_blockmodel_dl(g)
    gt.mcmc_equilibrate(state, wait=1000, mcmc_args=dict(niter=10))

    bs = [] # collect some partitions
    def collect_partitions(s):
        global bs
        bs.append(s.b.a.copy())

    gt.mcmc_equilibrate(state, force_niter=10000, mcmc_args=dict(niter=10),
                        callback=collect_partitions)

    # # Disambiguate partitions and obtain marginals
    pmode = gt.PartitionModeState(bs, converge=True)
    pv = pmode.get_marginal(u)
    state.draw(pos=pos, output="output/graph_alternate.png", bg_color="white", output_size=(1000, 1000),
               vertex_shape="pie", vertex_pie_fractions=pv, vertex_size=gt.prop_to_size(g.vp.vb, 3, 15, power=.3),
               edge_pen_width=gt.prop_to_size(g.ep.weight, 0.1, 0.15, power=2)
               )

In [ ]:
if not use_nx:
    u = gt.extract_largest_component(g)
    state = gt.minimize_blockmodel_dl(u)
    state.draw(pos=pos, output="output/graph_clusters.png", bg_color="white", output_size=(1000, 1000),
               vertex_size=gt.prop_to_size(g.vp.vb, 3, 15, power=.3),
               edge_pen_width=gt.prop_to_size(g.ep.weight, 0.1, 0.15, power=2))

# 4. Wordcloud
We can generate term frequencies with the unigram model:

In [ ]:
from textacy.representations.vectorizers import GroupVectorizer

vectorizer = vectorizer = GroupVectorizer(tf_type="linear", idf_type="smooth", norm="l2", min_df=1, max_df=0.95)
group_term_matrix = vectorizer.fit_transform(corpus_tokens, types)
largest = np.argsort(group_term_matrix.toarray(), axis=1)
for i, doc_type_largest in enumerate(largest):
    print(f"Document type: {vectorizer.grps_list[i]}")
    print([vectorizer.terms_list[j] for j in doc_type_largest[::-1][:20]])

In [ ]:
for i, doc_type_score in enumerate(group_term_matrix.toarray()):
    doc_to_score = dict(zip(vectorizer.terms_list, doc_type_score))
    wordcloud = wc.WordCloud(width=1000, height=1000, background_color="white",
                                colormap="tab10", random_state=2).generate_from_frequencies(doc_to_score)
    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(f"output/{field}/wordcloud-{vectorizer.grps_list[i]}.pdf")
    plt.show()

# 5. Topic Embedding Comparison

In the following, we take both corpora and model them together to understand which topics are shared and which ones different between the two.

In [ ]:
# Get embeddings and topic model for all data
assert load_all, "Must load all data for subsequent analysis"

output_path = os.path.join("output", "combined")
if not os.path.exists(output_path):
    os.makedirs(output_path)

topic_model, embeddings = model_topics(docs, dates, output_path, min_cluster_size=8)

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

# Plot documents and topics for all data
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='euclidean').fit_transform(embeddings)

# Convert data to DataFrame for easy plotting

df = pd.DataFrame(reduced_embeddings, columns=["x", "y"])
df["Type"] = [t == "safety" for t in types]
model = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
contour = px.density_contour(df, x="x", y="y", color="Type")
contour.update_traces(hovertemplate=None, hoverlabel=None)
fig = go.Figure(data=contour.data + model.data, layout=model.layout)
fig.write_html(os.path.join(output_path, "documents-combined.html"), include_mathjax = 'cdn')
fig.show()

In [ ]:
import matplotlib.patches
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay

sns.set_style("white")
X = df.drop(columns="Type").values
y = df["Type"].values
# classifier = RandomForestClassifier(n_estimators=100, random_state=42).fit(X, y)
classifier = SVC(kernel="rbf", random_state=42).fit(X, y)
disp = DecisionBoundaryDisplay.from_estimator(
    classifier, X, response_method="decision_function",
    xlabel="Dimension 1", ylabel="Dimension 2",
    alpha=0.5, cmap="twilight_shifted"
)
ax = disp.ax_
for t in ["safety", "ethics"]:
    ix = [tt == t for tt in types]
    ax.scatter(X[ix, 0], X[ix, 1], c="lightsteelblue" if t == "ethics" else "darksalmon", edgecolor="k", label=t.capitalize())
ax.legend(title="Classes")
# handles = [matplotlib.patches.Circle((0, 0), .0001, color="lightsteelblue", edgecolor="k", label="Ethics"),
        #    matplotlib.patches.Circle((0, 0), .0001, color="darksalmon", edgecolor="k", label="Safety")]
# legend = ax.legend(loc="lower left", title="Classes")
# ax.add_artist(legend)
ax.get_figure().tight_layout() 
ax.get_figure().savefig(os.path.join(output_path, "decision_boundary.pdf"))

# 6. Venue Analysis

In this section, we extract and visualise the major venues for AI safety papers. As our AI ethics papers come from FAccT and AIES proceedings, we do not do this for the AI ethics corpus.

In [ ]:
import time
import tqdm
import urllib

# Create folder to save the data
output_dir = os.path.join("data", field, "dblp")
os.makedirs(output_dir, exist_ok=True)

# Retrieve publications from DBLP by title
dblp_data = {}
not_found = {}  
raised_error = {}
multiple_hits = {}
try:
    dblp_data = json.load(open(f"{output_dir}/dblp_data.json", "r", encoding="utf-8"))
    not_found = json.load(open(f"{output_dir}/not_found.json", "r", encoding="utf-8"))
    raised_error = json.load(open(f"{output_dir}/raised_error.json", "r", encoding="utf-8"))
    multiple_hits = json.load(open(f"{output_dir}/multiple_hits.json", "r", encoding="utf-8"))
except FileNotFoundError as e:
    print(e)
BASE_API_URL = "https://dblp.org/search/publ/api"

We can now retrieve all publication information from DBLP. Note, these may be incomplete so further processing by hand is required. However, the downloaded data is already located in the data folder so running this cell should not be required.

In [ ]:
for paper in (pbar := tqdm.tqdm(safety)):
    title = paper['title']
    pid = paper['id']

    if pid in dblp_data or pid in not_found or pid in raised_error or pid in multiple_hits:
        continue

    request = f"{BASE_API_URL}?q={title}&format=json"
    request = urllib.parse.quote(request, safe=':/?&=')

    try:
        data = json.load(urllib.request.urlopen(request))
    except urllib.error.HTTPError as e:
        if e.code == 429:
            sleep_time = int(e.headers['Retry-After'])
            print(f"Rate limit exceeded. Sleeping for {sleep_time + 1} seconds.")
            time.sleep(sleep_time + 1.)
            data = json.load(urllib.request.urlopen(request))
        else:
            raised_error.append(pid)
            desc = f"Error retrieving data for {title}: {e}"
    
    hits = int(data['result']['hits']['@total'])
    if hits == 0:
        not_found.append(pid)
        desc = f"Title '{title}' not found"
    elif hits == 1:
        data = data['result']['hits']['hit'][0]
        dblp_data[pid] = data
        desc = f"Retrieved data for '{title}'"
    else:
        not_corr = [hit for hit in data['result']['hits']['hit'] if 'venue' in hit['info'] and hit['info']['venue'] != "CoRR"]
        if len(not_corr) == 1:
            dblp_data[pid] = not_corr[0]
            desc = f"Retrieved data for '{title}'"        
        else:
            desc = f"Multiple hits for '{title}': {hits}"
            multiple_hits[pid] = data

    json.dump(dblp_data, open(os.path.join(output_dir, "dblp_data.json"), "w", encoding="utf-8"), indent=2)
    json.dump(not_found, open(os.path.join(output_dir, "not_found.json"), "w", encoding="utf-8"), indent=2)
    json.dump(raised_error, open(os.path.join(output_dir, "raised_error.json"), "w", encoding="utf-8"), indent=2)
    json.dump(multiple_hits, open(os.path.join(output_dir, "multiple_hits.json"), "w", encoding="utf-8"), indent=2)

    pbar.set_postfix({"result": desc})

    time.sleep(1.)


In [ ]:
# Load DBLP data
dblp_data = json.load(open(os.path.join(output_dir, "dblp_data.json"), "r", encoding="utf-8"))
venues = pd.Series([data['info']['venue'] for data in dblp_data.values()])
counts = venues.value_counts().sort_values(ascending=False)

# Plot the distribution of the top 20 most frequent venues
plt.figure(figsize=(8, 5))
plt.barh(counts.index[:20], counts.values[:20], color="tab:blue")
plt.xlabel("Venue")
plt.ylabel("Frequency")
plt.title("Top 20 Venues")
plt.tight_layout()
plt.savefig(f"output/{field}/venues.pdf")
plt.show()